# IE 7275_HW3_Lingxuan_Ye

## Imports

In [ ]:
import gurobipy as gp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as shc
from gurobipy import GRB
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

## Problem 3

Read data.

In [ ]:
data = pd.read_csv('Cereals-3.csv').dropna()
# data.set_index('name', inplace=True)
name = data['name'].reset_index(drop=True)
data.drop('name', axis=1, inplace=True)
cat = data.select_dtypes(exclude='number')
data = data.assign(**{
    label: pd.Categorical(col).codes
    for label, col in cat.items()
})
data

In [ ]:
normalized = pd.DataFrame(normalize(data), columns=data.columns)

### 1

In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Dendrograms")
dend_single = shc.dendrogram(shc.linkage(normalized, method='single'))
plt.axhline(color='r', linestyle='--')
ticks, labels = plt.xticks()
plt.xticks(ticks, name.loc[[int(i.get_text()) for i in labels]])
plt.show()

In [ ]:
single = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single')
normalized['cluster'] = single.fit_predict(normalized)
sin_df = normalized.groupby('cluster').mean()

In [ ]:
plt.figure(figsize=(10, 7))
plt.title("Dendrograms")
dend_complete = shc.dendrogram(
    shc.linkage(normalized.drop('cluster', axis=1), method='complete')
)
plt.axhline(color='r', linestyle='--')
ticks, labels = plt.xticks()
plt.xticks(ticks, name.loc[[int(i.get_text()) for i in labels]])
plt.show()

In [ ]:
complete = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='complete')
normalized['cluster'] = complete.fit_predict(normalized)
com_df = normalized.groupby('cluster').mean()

In [ ]:
sin_df - com_df

The most observations are clustered together from single linage so it performs worse than complete linkage.

The centroids in

- sodium
- potass
- rating

significantly varies.

### 2

Complete linkage for its more balanced in each clusters.

### 3

Choose complete linkage.

- 3 clusters
- cut-off = 0.9

### 4

The amount of fat and sugar should be low and because of their quantitatively small range of values, the data should be **normalized** to decrease other variables' contibutions when clustering. 

In [ ]:
com_df

Cluster 2 in complete linkage should be used.

## Problem 5

In [ ]:
pip install mlxtend gurobipy -q

### a

lhs: rule body

rhs: rule head

- confidence: percentage of the rule head occurence amongst all groups.

    $$
    \begin{align*}
    {confidence}&=\frac{support}{number\ of\ cases}
    \end{align*}
    $$

- support: percentage of groups containing all items listed in association rule.

    $$
    \begin{align*}
    {support}&=\frac{\sum{occurence\{Blush,\ Concealer,\ Mascara,\ Eye.shadow,\ lipstick,\ Eyebrow.Pencils\}}}{support\{Eyebro.Pencils\}}
    \end{align*}
    $$

- lift: ratio of confidence and expected confidence.

    $$
    \begin{align*}
    {lift}&=\frac{confidence}{\cfrac{support\{Blush,\ Concealer,\ Mascara,\ Eye.shadow,\ lipstick\}\times support\{Eyebro.Pencils\}}{support\{Blush,\ Concealer,\ Mascara,\ Eye.shadow,\ lipstick\}}}
    \end{align*}
    $$


### b

In [ ]:
data = pd.read_csv('Cosmetics.csv').set_index('Trans. ')
data = data.iloc[:, [-1, *range(data.shape[1]-1)]]
data

In [ ]:
freq = apriori(data, min_support=0.1, use_colnames=True)
freq

#### 1

Top 20 by lift:

In [ ]:
inst = association_rules(freq, min_threshold=0.7)
inst.nlargest(20, 'lift')

Top 20 by confidence

In [ ]:
inst.nlargest(20, 'confidence')

#### 2

Association rule with high confidence are much likes to have high lift, therefore we can observe many repeat records which are redundant. As we can see, the `Eye.shadow` and `Mascara` are rule heads in the most records which means they are more likely to be purchased with others, so we can sightly decrease the prices of other irrelevant products and increase those two above.

## Problem 7

Table-8 comes from an excel-generated `.csv` file.

In [ ]:
data = pd.read_csv('Table-8.csv').set_index('student', drop=True)

### a

In [ ]:
data * data.loc['EN']

As we can see, LN, MH, JH, DU and DS have at least one class the same with EN.

In [ ]:
STU = ['SQL', 'DM in R', 'R Prog', 'Regression']

In [ ]:
mean = data[data != 0].mean(axis=1)
mean

### b

The single course should be recommended to EN Spatial based on the single nearst student to him, which is DU.

### c

In [ ]:
cos_sim = pd.DataFrame(cosine_similarity(data), index=data.index, columns=data.index)
cos_sim

### d

In [ ]:
cos_sim['EN']

### e

In [ ]:
df = pd.DataFrame(cosine_similarity(data.T), index=data.columns, columns=data.columns)
df.drop(STU, axis=1, inplace=True)
df

In [ ]:
(df * np.array(data.loc['EN']).reshape(9, 1)).sum()


Python is the recommended cource.